# Phase 1 LangGraph Walkthrough

This notebook teaches the LangGraph migration step by step while importing real code from `execution/langgraph/`.

## Run Order

If you changed Python files, restart kernel first. Then run cells top-to-bottom so imports and state are fresh.

In [ ]:
from pathlib import Path
import sys

def _candidate_roots() -> list[Path]:
    cwd = Path.cwd().resolve()
    candidates = [cwd, *cwd.parents, Path('/home/nir/dev/agent_phase0')]
    seen = set()
    unique = []
    for c in candidates:
        if str(c) in seen:
            continue
        seen.add(str(c))
        unique.append(c)
    return unique

repo_root = None
for candidate in _candidate_roots():
    if (candidate / 'execution' / 'langgraph' / 'graph.py').exists():
        repo_root = candidate
        break

if repo_root is None:
    raise RuntimeError('Could not locate repo root containing execution/langgraph/graph.py')

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from execution.langgraph.graph import LangGraphOrchestrator
from execution.langgraph.memo_store import SQLiteMemoStore
from execution.langgraph.policy import MemoizationPolicy

print('repo_root =', repo_root)

## 1) Initialize orchestrator

Use OpenAI when `OPENAI_API_KEY` is configured; otherwise provider falls back to Groq.

In [ ]:
store = SQLiteMemoStore('.tmp/memo_store.db')
policy = MemoizationPolicy(max_policy_retries=2)
orchestrator = LangGraphOrchestrator(memo_store=store, policy=policy)

## 2) Run a task

The graph records tool calls, enforces memoization for heavy deterministic writes, and returns a final summary.

In [ ]:
prompt = '''Please complete these tasks in order:
1) repeat message: Agent loop is working!
2) sort numbers 5,2,8,1,9,3 ascending
3) uppercase: the quick brown fox
4) write fibonacci up to 100th number to fib.txt
Then finish with a summary.'''
result = orchestrator.run(prompt)
result['run_id'], result['answer']

## 3) Debug run state

This shows retry counters and per-tool usage (including duplicate-call protection counts).

In [ ]:
debug_state = {
    'retry_counts': result['state'].get('retry_counts', {}),
    'tool_call_counts': result['state'].get('tool_call_counts', {}),
    'memo_events_count': len(result.get('memo_events', [])),
    'checkpoints_count': len(result.get('checkpoints', [])),
}
debug_state

## 4) Inspect memoization state

Use keys suggested by policy (for fibonacci writes this is usually `write_file:fib.txt`).

In [ ]:
lookup = store.get(run_id=result['run_id'], key='write_file:fib.txt', namespace='run')
lookup